In [1]:
from src.defaults import *
from src.datasets.novelty import *

In [2]:
# download_models_from_neptune("NLI-81")
field = load_field('NLI-81')
# field = None


dataset_conf = {'dataset': 'dlnd', 'max_num_sent': 60,"sent_tokenizer":"spacy","batch_size":4,"device":"cuda"}
# dataset_conf = {'dataset': 'dlnd', 'max_num_sent': 50,"sent_tokenizer":"spacy", "tokenizer":'spacy',"max_len":50,"batch_size":32,"device":"cuda"}
model_conf = {'results_dir': 'results', 'device': 'cuda', 'dropout': 0.3, 'dataset': 'dlnd', 'hidden_size': 400, 'use_glove': False, "max_num_sent": 60}

In [3]:
data = dlnd(dataset_conf,sentence_field = field)


In [4]:
for i in data.train_iter:
    print(i)
    break


[torchtext.data.batch.Batch of size 4]
	[.source]:[torch.cuda.LongTensor of size 4x60x50 (GPU 0)]
	[.target]:[torch.cuda.LongTensor of size 4x60x50 (GPU 0)]
	[.label]:[torch.cuda.LongTensor of size 4 (GPU 0)]


In [5]:
from src.model.nli_models import *
from src.model.novelty_models import *



def load_encoder1(enc_data):
    if enc_data['options'].get("attention_layer_param",0)==0:
        model = bilstm_snli(enc_data["options"])
    elif enc_data['options'].get("r",0)==0:
        model = attn_bilstm_snli(enc_data["options"])
    else:
        model = struc_attn_snli(enc_data["options"])
    return model

def load_encoder(_id):
    model_path = os.path.join('./results/',_id,"model.pt")
    model_data = torch.load(model_path)
    return model_data

model_data = load_encoder('NLI-81')

model_conf["encoder_dim"] = model_data["options"]["hidden_size"]
model_data['options']["use_glove"] = False

model = attn_bilstm_snli(model_data['options'])

# dan = DAN(model_conf,model.encoder)

# dan = dan.cuda()

In [9]:
ens = FusionClassifier_novelty(estimator=DAN,n_estimators=2,cuda=True,estimator_args={"conf":model_conf,"encoder":model.encoder})

In [10]:
ens.set_optimizer("Adam", lr=0.001)

In [11]:
train_loader = data.train_iter
val_loader = data.val_iter
set_logger("ensemble_dev")
ens.fit(train_loader,
          epochs=2,
          test_loader=val_loader)
    

Log will be saved in '/content/NoveltyDetectionResearch/logs'.
Start logging into file /content/NoveltyDetectionResearch/logs/ensemble_dev-2021_02_16_00_57.log...
2021-02-16 00:57:49,585 - INFO: Epoch: 000 | Batch: 000 | Loss: 0.74848 | Correct: 1/4
2021-02-16 00:58:13,109 - INFO: Epoch: 000 | Batch: 100 | Loss: 0.65103 | Correct: 3/4
2021-02-16 00:58:36,602 - INFO: Epoch: 000 | Batch: 200 | Loss: 0.68120 | Correct: 2/4
2021-02-16 00:59:00,119 - INFO: Epoch: 000 | Batch: 300 | Loss: 0.69258 | Correct: 3/4
2021-02-16 00:59:23,627 - INFO: Epoch: 000 | Batch: 400 | Loss: 0.71224 | Correct: 0/4
2021-02-16 00:59:47,094 - INFO: Epoch: 000 | Batch: 500 | Loss: 0.68313 | Correct: 3/4
2021-02-16 01:00:10,633 - INFO: Epoch: 000 | Batch: 600 | Loss: 0.66220 | Correct: 3/4
2021-02-16 01:00:34,178 - INFO: Epoch: 000 | Batch: 700 | Loss: 0.69657 | Correct: 2/4
2021-02-16 01:00:57,686 - INFO: Epoch: 000 | Batch: 800 | Loss: 0.69475 | Correct: 2/4
2021-02-16 01:01:21,197 - INFO: Epoch: 000 | Batch: 90

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torchensemble._base import BaseModule, torchensemble_model_doc
from torchensemble.utils import io
from torchensemble.utils import set_module
from torchensemble.utils import operator as op
from torchensemble.utils.logging import set_logger


@torchensemble_model_doc("""Implementation on the FusionClassifier.""", "model")
class FusionClassifier_novelty(BaseModule):
    def _forward(self, x, y):
        """
        Implementation on the internal data forwarding in FusionClassifier.
        """
        # Average
        outputs = [estimator(x, y) for estimator in self.estimators_]
        output = op.average(outputs)

        return output

    @torchensemble_model_doc(
        """Implementation on the data forwarding in FusionClassifier.""",
        "classifier_forward",
    )
    def forward(self, x, y):
        output = self._forward(x, y)
        proba = F.softmax(output, dim=1)

        return proba

    @torchensemble_model_doc(
        """Set the attributes on optimizer for FusionClassifier.""", "set_optimizer"
    )
    def set_optimizer(self, optimizer_name, **kwargs):
        self.optimizer_name = optimizer_name
        self.optimizer_args = kwargs

    @torchensemble_model_doc(
        """Set the attributes on scheduler for FusionClassifier.""", "set_scheduler"
    )
    def set_scheduler(self, scheduler_name, **kwargs):
        self.scheduler_name = scheduler_name
        self.scheduler_args = kwargs
        self.use_scheduler_ = True

    @torchensemble_model_doc(
        """Implementation on the training stage of FusionClassifier.""", "fit"
    )
    def fit(
        self,
        train_loader,
        epochs=100,
        log_interval=100,
        test_loader=None,
        save_model=True,
        save_dir=None,
    ):

        # Instantiate base estimators and set attributes
        for _ in range(self.n_estimators):
            self.estimators_.append(self._make_estimator())
        self._validate_parameters(epochs, log_interval)
        self.n_outputs = 2
        optimizer = set_module.set_optimizer(
            self, self.optimizer_name, **self.optimizer_args
        )

        # Set the scheduler if `set_scheduler` was called before
        if self.use_scheduler_:
            self.scheduler_ = set_module.set_scheduler(
                optimizer, self.scheduler_name, **self.scheduler_args
            )

        # Utils
        criterion = nn.CrossEntropyLoss()
        best_acc = 0.0

        # Training loop
        for epoch in range(epochs):
            self.train()
            for batch_idx, (batch) in enumerate(train_loader):

                source, target, label = (
                    batch.source.to(self.device),
                    batch.target.to(self.device),
                    batch.label.to(self.device),
                )

                # data, target = data.to(self.device), target.to(self.device)

                optimizer.zero_grad()
                output = self._forward(source, target)
                loss = criterion(output, label)
                loss.backward()
                optimizer.step()

                # Print training status
                if batch_idx % log_interval == 0:
                    with torch.no_grad():
                        _, predicted = torch.max(output.data, 1)
                        correct = (predicted == label).sum().item()

                        msg = (
                            "Epoch: {:03d} | Batch: {:03d} | Loss:"
                            " {:.5f} | Correct: {:d}/{:d}"
                        )
                        self.logger.info(
                            msg.format(epoch, batch_idx, loss, correct, label.size(0))
                        )

            # Validation
            if test_loader:
                self.eval()
                with torch.no_grad():
                    correct = 0
                    total = 0
                    for _, (batch) in enumerate(test_loader):

                        source, target, label = (
                            batch.source.to(self.device),
                            batch.target.to(self.device),
                            batch.label.to(self.device),
                        )

                        output = self.forward(source, target)
                        _, predicted = torch.max(output.data, 1)
                        correct += (predicted == label).sum().item()
                        total += label.size(0)
                    acc = 100 * correct / total

                    if acc > best_acc:
                        best_acc = acc
                        if save_model:
                            io.save(self, save_dir, self.logger)

                    msg = (
                        "Epoch: {:03d} | Validation Acc: {:.3f}"
                        " % | Historical Best: {:.3f} %"
                    )
                    self.logger.info(msg.format(epoch, acc, best_acc))

            # Update the scheduler
            if hasattr(self, "scheduler_"):
                self.scheduler_.step()

        if save_model and not test_loader:
            io.save(self, save_dir, self.logger)

    @torchensemble_model_doc(
        """Implementation on the evaluating stage of FusionClassifier.""",
        "classifier_predict",
    )
    def predict(self, test_loader):
        self.eval()
        correct = 0
        total = 0

        for _, (batch) in enumerate(test_loader):
            source, target, label = (
                batch.source.to(self.device),
                batch.target.to(self.device),
                batch.label.to(self.device),
            )
            output = self.forward(source,target)
            _, predicted = torch.max(output.data, 1)
            correct += (predicted == label).sum().item()
            total += label.size(0)

        acc = 100 * correct / total

        return acc



In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from torchvision import datasets, transforms

from torchensemble import VotingClassifier
from torchensemble.utils.logging import set_logger

# Define Your Base Estimator
class MLP(nn.Module):

    def __init__(self):
        super(MLP, self).__init__()

        self.linear1 = nn.Linear(784, 128)
        self.linear2 = nn.Linear(128, 128)
        self.linear3 = nn.Linear(128, 10)

    def forward(self, X):
        X = X.view(X.size(0), -1)
        output = F.relu(self.linear1(X))
        output = F.dropout(output)
        output = F.relu(self.linear2(output))
        output = self.linear3(output)

        return output

# Load MNIST dataset
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])

train = datasets.MNIST('../../Dataset', train=True, download=True, transform=transform)
test = datasets.MNIST('../../Dataset', train=False, transform=transform)
train_loader = torch.utils.data.DataLoader(train, batch_size=128, shuffle=True)
test_loader = torch.utils.data.DataLoader(test, batch_size=128, shuffle=True)

# Set the Logger
logger = set_logger("classification_mnist_mlp")

# Set the model
model = VotingClassifier(
    estimator=MLP,
    n_estimators=10,
    cuda=True
)
model.set_optimizer("Adam", lr=1e-3, weight_decay=5e-4)

# Train and Evaluate
model.fit(train_loader,
          epochs=50,
          test_loader=test_loader)

Extracting ../../Dataset/MNIST/raw/train-images-idx3-ubyte.gz to ../../Dataset/MNIST/raw


Extracting ../../Dataset/MNIST/raw/train-labels-idx1-ubyte.gz to ../../Dataset/MNIST/raw


Extracting ../../Dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to ../../Dataset/MNIST/raw


Extracting ../../Dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../../Dataset/MNIST/raw
Processing...
Done!



Log will be saved in '/content/NoveltyDetectionResearch/logs'.
Create folder 'logs/'
Start logging into file /content/NoveltyDetectionResearch/logs/classification_mnist_mlp-2021_02_15_21_55.log...
Estimator: 000 | Epoch: 000 | Batch: 000 | Loss: 2.30158 | Correct: 18/128
Estimator: 000 | Epoch: 000 | Batch: 100 | Loss: 0.51601 | Correct: 105/128
Estimator: 000 | Epoch: 000 | Batch: 200 | Loss: 0.33963 | Correct: 113/128
Estimator: 000 | Epoch: 000 | Batch: 300 | Loss: 0.33299 | Correct: 119/128
Estimator: 000 | Epoch: 000 | Batch: 400 | Loss: 0.18297 | Correct: 121/128
Estimator: 001 | Epoch: 000 | Batch: 000 | Loss: 2.31871 | Correct: 13/128
Estimator: 001 | Epoch: 000 | Batch: 100 | Loss: 0.47951 | Correct: 108/128
Estimator: 001 | Epoch: 000 | Batch: 200 | Loss: 0.47146 | Correct: 108/128
Estimator: 001 | Epoch: 000 | Batch: 300 | Loss: 0.26730 | Correct: 119/128
Estimator: